<a href="https://colab.research.google.com/github/RodGuarneros/BigData_Frameworks/blob/main/demographicsII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the version of spark 3.0 form http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
#spark_version 3.0.2
spark_version = "spark-3.1.1"
os.environ["SPARK_VERSION"] = spark_version

# Install Java, Spark, and Findspark
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [53.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:13 https://developer.download.nvi

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demographicsFilter").getOrCreate()

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/dataviz-curriculum/day_1/demographics.csv"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("demographics.csv"), inferSchema=True, sep=',')

# Show DataFrame
df.show()

+---+--------------------+---+------------+---------+--------+------------------+---------------+------+-------------+
| id|                name|age|height_meter|weight_kg|children|        occupation|academic_degree|salary|     location|
+---+--------------------+---+------------+---------+--------+------------------+---------------+------+-------------+
|  0|       Darlena Avila| 58|        1.87|       53|       1|     Choreographer|            PhD|    68| South Dakota|
|  1|            Yan Boyd| 65|         1.8|       40|       0|         Cellarman|       Bachelor|    73|     Delaware|
|  2|         Joette Lane| 32|         1.8|       73|       1|Veterinary Surgeon|         Master|    69| South Dakota|
|  3|        Jazmine Hunt| 61|        1.79|       89|       0|            Hawker|            PhD|    88|    Louisiana|
|  4|      Remedios Gomez| 23|        1.64|       51|       2|     Choreographer|       Bachelor|    83|West Virginia|
|  5|        Myung Brewer| 20|        1.68|     

In [5]:
# What occupation had the highest salary?
df.orderBy(df["Salary"].desc()).select("occupation", "Salary").limit(3).show()

+----------------+------+
|      occupation|Salary|
+----------------+------+
|    Metallurgist|    90|
| Plant Attendant|    90|
|Parts Supervisor|    90|
+----------------+------+



In [6]:
# What occupation had the lowest salary?
df.orderBy(df["Salary"]).select("occupation", "Salary").limit(2).show()

+-----------------+------+
|       occupation|Salary|
+-----------------+------+
|Lighthouse Keeper|    65|
|   Window Dresser|    65|
+-----------------+------+



In [7]:
# What is the mean salary of this dataset?
from pyspark.sql.functions import mean
df.select(mean("Salary")).show()

+-----------+
|avg(Salary)|
+-----------+
|     77.738|
+-----------+



In [ ]:
# What is the max and min of the Salary column?
from pyspark.sql.functions import max, min
df.select(max("Salary"), min("Salary")).show()

+-----------+-----------+
|max(Salary)|min(Salary)|
+-----------+-----------+
|         90|         65|
+-----------+-----------+



In [9]:
# Show all of the occupations where salaries were above 80k
from pyspark.sql.functions import count
df.filter("Salary > 80").select("occupation").show(35)

+--------------------+
|          occupation|
+--------------------+
|              Hawker|
|       Choreographer|
|          Millwright|
|   Medical Physicist|
|           Scientist|
|     Claims Adjustor|
| Planning Technician|
|       Booking Clerk|
|      Sub-Postmaster|
|        Shelf Filler|
|             Chemist|
|        Betting Shop|
|     Hire Car Driver|
|    Heating Engineer|
|    Vehicle Assessor|
|   Building Surveyor|
|Advertising Contr...|
|   Medical Physicist|
|            Labourer|
|   Technical Analyst|
|             Actress|
|        Share Dealer|
|      Prison Officer|
|           Rug Maker|
|   Lighthouse Keeper|
|          Shipwright|
|   Marketing Manager|
| Insurance Inspector|
|        Share Dealer|
|      Pipe Inspector|
|Golf Club Profess...|
|              Seaman|
|       Stock Manager|
|         Lock Keeper|
|        Grave Digger|
+--------------------+
only showing top 35 rows



In [10]:
# BONUS
# What is the average age and height for each academic degree type?
# HINT: You will need to use `groupby` to solve this
avg_df = df.groupBy("academic_degree").avg()
avg_df.select("academic_degree", "avg(age)", "avg(height_meter)").show()

+---------------+------------------+------------------+
|academic_degree|          avg(age)| avg(height_meter)|
+---------------+------------------+------------------+
|            PhD| 43.15976331360947|1.7438165680473379|
|         Master|43.139318885448915|1.7549226006191951|
|       Bachelor| 42.51032448377581| 1.757227138643069|
+---------------+------------------+------------------+



In [11]:
# BONUS
# What is the average salary for each academic degree type?
# HINT: You will need to use `groupby` to solve this
avg_df = df.groupBy("academic_degree").avg()
avg_df.select("academic_degree", "avg(salary)").show()

+---------------+-----------------+
|academic_degree|      avg(salary)|
+---------------+-----------------+
|            PhD|77.88757396449704|
|         Master|77.94736842105263|
|       Bachelor|77.38938053097345|
+---------------+-----------------+

